### Day 14: Part 2

Changes:
- bitmask bit of `0` is now ignored
- bitmask bit of `1` still overwrites
- bitmask bit of `x` is just going to mean all combinations are possible....

In [26]:
def bitmask2(mask, intVal):
    """
    Handle mask & integer, apply mask, and output integer
    Build out all possible combinations of floating bits (which can be 1 or 0)
    """
    converted_val = bin(intVal + 2**36)
    change_val = converted_val[3:]
    print(change_val)
    new_val = ''
    for i in range(len(mask)):
        if mask[i] == '1':
            new_val = new_val + '1'
        elif mask[i] == 'X':
            new_val = new_val + 'F' # makes it easier to find floating
        else:
            new_val = new_val + change_val[i]
    return new_val

In [27]:
# ex 1: 
mask = "000000000000000000000000000000X1001X"
integerVal = 100
memory_address = 42
bitmask2(mask, memory_address)

000000000000000000000000000000101010


'000000000000000000000000000000F1101F'

In [28]:
# ex 2: 
mask = "00000000000000000000000000000000X0XX"
memory_address = 26
bitmask2(mask, memory_address)

000000000000000000000000000000011010


'00000000000000000000000000000001F0FF'

### Now need a clean way to expand out for floating bits

In [47]:
# Very cool:
# https://stackoverflow.com/questions/4928297/all-permutations-of-a-binary-sequence-x-bits-long
["".join(seq) for seq in product("01", repeat=3)]

['000', '001', '010', '011', '100', '101', '110', '111']

In [94]:
from itertools import product
import math

def expansionFunc(inputstr):
    """Depending on number of Fs, we will have 2^TOT_F changes"""
    tot_x = inputstr.count('F')
    new_adds = ["".join(seq) for seq in product("01", repeat=int(tot_x))]
    print(new_adds)
    # to do: may want to sum these outputs instead of just outputting a list
    output_list = []
    for i in range(len(new_adds)):
        new_val = ''
        idx = 0
        for j in range(len(inputstr)):
            if inputstr[j] == 'F':
                new_val = new_val + new_adds[i][idx]
                idx += 1
            else:
                new_val = new_val + inputstr[j]
        
        # append
        output_list.append(new_val)
    
    return output_list

In [95]:
# ex 1:
expansionFunc("000000000000000000000000000000F1101F")

['00', '01', '10', '11']


['000000000000000000000000000000011010',
 '000000000000000000000000000000011011',
 '000000000000000000000000000000111010',
 '000000000000000000000000000000111011']

In [96]:
# ex 2:
expansionFunc("00000000000000000000000000000001F0FF")

['000', '001', '010', '011', '100', '101', '110', '111']


['000000000000000000000000000000010000',
 '000000000000000000000000000000010001',
 '000000000000000000000000000000010010',
 '000000000000000000000000000000010011',
 '000000000000000000000000000000011000',
 '000000000000000000000000000000011001',
 '000000000000000000000000000000011010',
 '000000000000000000000000000000011011']